<style>
/* Set font size for headers */
h1, h2, h3, h4, h5, h6 {
    font-size: 32px; /* Adjust as needed */
}

/* Set font size for paragraphs */
p, li, ul {
    font-size: 24px; /* Adjust as needed */
}
</style>


Introduction to the Julia programming language

![](images/julia-logo.svg)

## 20 Symbolic Math

## Symbolic math in Julia

Julia offers several packages for symbolic computer algebra, providing functionalities for symbolic manipulation, differentiation, integration, equation solving, and more. Here we focus on [Symbolics.jl](https://symbolics.juliasymbolics.org/stable/) and [SymPy.jl](https://juliapackages.com/p/sympy).

[Symbolics.jl](https://symbolics.juliasymbolics.org/stable/) is a fast and modern Computer Algebra System (CAS) for a fast and modern programming language (Julia). The goal is to have a high-performance and parallelized symbolic algebra system that is directly extendable in the same language as that of the users.

[SymPy.jl](https://juliapackages.com/p/sympy) provides a Julia interface to the SymPy library in Python, which is a well-established symbolic mathematics library. It allows you to perform various symbolic operations including symbolic manipulation, solving equations, calculus, and more.

## Derivatives

In [2]:
using Symbolics

In [3]:
@variables t
D = Differential(t)

z = t + t^2
D(z)

Differential(t)(t + t^2)

D is a lazy operator because it lets us symbolically represent "The derivative with respect to $t$". If we want to expand the derivative operators, we'd use expand_derivatives:

In [4]:
expand_derivatives(D(z))

1 + 2t

The variable, that you are taking the derivative with respect to, is accessed with:

In [5]:
D.x

t

## Simplification and Substitution

In [6]:
@variables x y
simplify(2x + 2y)

2(x + y)

In [7]:
simplify(sin(x)^2+cos(x)^2)

1

In [8]:
simplify(cosh(x)^2-sinh(x)^2)

1

This can be applied to arrays by using Julia's broadcast mechanism:

In [9]:
B = simplify.([t + t^2 + t + t^2  2t + 4t
               x + y + y + 2t     x^2 - x^2 + y^2])

2×2 Matrix{Num}:
   2(t + t^2)   6t
 2(t + y) + x  y^2

We can then use substitute to change values of an expression around:

In [10]:
simplify.(substitute.(B, (Dict(x => y^2),)))

2×2 Matrix{Num}:
     2(t + t^2)   6t
 2(t + y) + y^2  y^2

## Example: Error propagation with Symbolics.jl

In [11]:
function err_prop(f, vars)
    @variables sum
    sum = 0
    for (x, sigma) in vars
        D = Differential(x)
        sum += expand_derivatives(D(f))^2 * sigma^2 
    end
    return sqrt(simplify(sum))
end

err_prop (generic function with 1 method)

Take the volume of a cylinder as an example:

In [12]:
@variables r, h, σ_r, σ_h
V = pi * r^2 * h
σ_V = err_prop(V, [(r, σ_r), (h, σ_h)])

sqrt(39.47841760435743(h^2)*(r^2)*(σ_r^2) + 9.869604401089358(r^4)*(σ_h^2))

Plug in numerical values:

In [13]:
using Formatting

r_meas = 3 # cm
σ_r_meas = 0.1 # cm
h_meas = 5 # cm
σ_h_meas = 0.1; # cm

V_num = substitute(V, Dict(r => r_meas, h => h_meas))
σ_V_num = substitute(σ_V, (Dict(r => r_meas, σ_r => σ_r_meas, 
                     h => h_meas, σ_h => σ_h_meas)))

printfmt("V = ({:.2f} ± {:.2f}) cm^3", Symbolics.value(V_num), Symbolics.value(σ_V_num))

V = (141.37 ± 9.84) cm^3

## Error propgation with Measurements.jl

For the particular problem of error propgation, there are also other tools, e.g., Measurements.jl

See [arXiv:1610.08716](https://arxiv.org/abs/1610.08716)

In [14]:
using Measurements

r = measurement(3, 0.1)
h = measurement(5, 0.1)
V = pi * r^2 * h

141.4 ± 9.8

## SymPy

In [15]:
using SymPy
@which simplify

SymbolicUtils

In [16]:
sympy.sqrt(2)

√2

In [17]:
sympy.sqrt(2).evalf(50)

1.4142135623730950488016887242096980785696718753769

In [18]:
x, y = sympy.symbols("x y")
x + 1/2

x + 0.5

In [19]:
x + sympy.S(1)/2

x + 1/2

## Expression manipulation

In [20]:
expr = sympy.cosh(x)^2 - sympy.sinh(x)^2

      2          2   
- sinh (x) + cosh (x)

In [21]:
sympy.simplify(expr)

1

In [22]:
sympy.trigsimp(2 * sin(x) * cos(x))

sin(2⋅x)

In [23]:
sympy.factor(x^2 +2x +1)

       2
(x + 1) 

## Differentiation

In [24]:
sympy.diff(sin(x)*exp(x), x)

 x           x       
ℯ ⋅sin(x) + ℯ ⋅cos(x)

## Symbolic integration

In [25]:
sympy.integrate(exp(x)*sin(x) + exp(x)*cos(x), x)

 x       
ℯ ⋅sin(x)

In [26]:
sympy.integrate(cos(x)^2, (x, 0, pi))

π
─
2

## Solving equations

In [27]:
sympy.solve(x^2 - 8x + 15, x)

2-element Vector{Sym{PyCall.PyObject}}:
 3
 5

In [36]:
eq1 = sympy.Eq(x + y, 5)
eq2 = sympy.Eq(x^2 + y^2, 17)
sympy.solve((eq1, eq2), (x, y))

2-element Vector{Tuple{Sym{PyCall.PyObject}, Sym{PyCall.PyObject}}}:
 (4, 1)
 (1, 4)

## Substitution

In [29]:
expr = x^2 + y^2
expr.subs(x, 2)

 2    
y  + 4

In [30]:
expr.subs([(x,3), (y,4)])

25

## Variable transformation

In [33]:
using Latexify
x, y, z, r, phi, theta = sympy.symbols("x, y, z, r, theta, phi")
(x, y, z) = (r * sin(theta) * cos(phi), 
             r * sin(theta) * sin(phi), 
             r * cos(theta))
X = sympy.Matrix([x, y, z])
Y = sympy.Matrix([r, theta, phi])
J = X.jacobian(Y)
latexify(J)

"\\begin{equation}\n\\left[\n\\begin{array}{ccc}\n\\sin\\left( \\phi \\right) \\cdot \\cos\\left( \\theta \\right) & r \\cdot \\cos\\left( \\phi \\right) \\cdot \\cos\\left( \\theta \\right) & \\left(  - r \\right) \\cdot \\sin\\left( \\phi \\right) \\cdot \\sin\\left( \\theta \\right) \\\\\n\\sin\\left( \\phi \\ri" ⋯ 19 bytes ⋯ "t( \\theta \\right) & r \\cdot \\sin\\left( \\theta \\right) \\cdot \\cos\\left( \\phi \\right) & r \\cdot \\sin\\left( \\phi \\right) \\cdot \\cos\\left( \\theta \\right) \\\\\n\\cos\\left( \\phi \\right) & \\left(  - r \\right) \\cdot \\sin\\left( \\phi \\right) & 0 \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

In [32]:
sympy.simplify(J.det())

 2       
r ⋅sin(φ)